In [32]:
import pandas as pd
import os

import re
def removeURLFromText(text):
    #print(str(text))
    text = re.sub(r'b"*', '', str(text), flags=re.MULTILINE)
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', str(text), flags=re.MULTILINE)
    text = re.sub(r'http?:\/\/.*[\r\n]*', '', str(text), flags=re.MULTILINE)
    #text = re.sub(r'"', '', str(text), flags=re.MULTILINE)
    text = re.sub(r'^"|"$', '', text)
    text = re.sub(r'\'', '', text)
    text = re.sub(r'[^a-zA-Z0-9 ]',r'',text)
    text = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",text).split())
    return (text)

def parseExcelFileWithMultipleSheetsAndCombine(excelFilePath,SheetsToParse):

    df = pd.ExcelFile(excelFilePath,encoding="utf-8",parse_dates=['created_at'])

    masterDf = df.parse("All")

    for index,sheet_name in enumerate(SheetsToParse):
        if index == 0:
            WholeDf = df.parse(sheet_name)
            WholeDf["STOCK"]= sheet_name
        else:
            tempDf = df.parse(sheet_name)
            tempDf["STOCK"]= sheet_name
            WholeDf= WholeDf.append(tempDf) #read a specific sheet to DataFrame

    WholeDf['text'] = WholeDf['text'].apply(lambda x: removeURLFromText(x))
    WholeDfNonNULL =WholeDf[WholeDf.Rating == WholeDf.Rating]
    
    return (WholeDfNonNULL)
    
SheetsToParse=['AAPL',
               'MSFT',
               'GE',
               'IBM',
               'DIS',
               'PG',
               'AXP',
               'BA',
               'DD',
               'JNJ',
               'KO',
               'MCD',
               'MMM']

WholeDf= parseExcelFileWithMultipleSheetsAndCombine("Tweet-Scale.xlsx",SheetsToParse)

In [42]:
def ConvertFiveCategoryTo3Sentiments(x):
    if x >= 5 or x == 4:
        return 'positive'
    if x == 2 or x == 1:
        return 'negative'
    return "neutral"
 
def categoryToNumber(x):
    if x == 'positive':
        return 3
    if x == 'negative':
        return 1
    return 2
WholeDf['Rating_m'] = WholeDf['Rating'].apply(lambda x: ConvertFiveCategoryTo3Sentiments(x))

In [34]:
secondDataFrame = pd.read_csv("twitterStockDataForSentimental.csv")

In [35]:
secondDataFrame.head()
secondDataFrame['id'] = secondDataFrame['UserID']
secondDataFrame['Rating_m'] = secondDataFrame['sentiment']
secondDataFrame['text'] = secondDataFrame['Tweet']
secondDataFrame['created_at'] = secondDataFrame['Time']




In [48]:
secondDataFrame1 = secondDataFrame[['text','Rating_m']]
WholeDataFrame1 = WholeDf[['text','Rating_m']]

secondDataFrame1 = secondDataFrame1.append(WholeDataFrame1)
secondDataFrame1['Rating_m'] = secondDataFrame1['Rating_m'].apply(lambda x: categoryToNumber(x))
secondDataFrame1.to_csv("twitter_training.csv")

In [49]:
import numpy as np
len((secondDataFrame1['Rating_m'] == secondDataFrame1['Rating_m']) == True)
len(secondDataFrame1['Rating_m'])
#np.nan(secondDataFrame1['Rating_m'])
for i in secondDataFrame1['Rating_m']:
    print(i)

2
2
3
1
3
2
1
2
2
3
3
1
2
2
1
2
1
3
3
1
2
1
2
2
1
1
2
2
1
3
2
1
2
3
1
2
2
1
1
2
1
2
3
3
2
1
1
2
2
3
3
1
3
2
2
3
2
2
2
2
3
2
2
3
3
2
2
2
2
2
2
2
3
2
3
1
3
2
3
2
2
2
2
2
2
2
3
2
3
2
2
3
2
2
3
2
1
2
2
1
2
2
2
2
3
2
2
3
2
2
2
2
1
2
2
2
1
2
1
3
3
2
2
2
3
3
2
1
2
3
1
2
2
3
3
2
2
1
1
1
3
2
2
1
2
3
1
2
2
1
1
3
3
2
2
2
2
2
2
1
3
2
2
3
1
3
2
2
3
2
3
2
3
3
3
3
2
2
2
3
3
3
3
2
3
2
2
2
2
3
2
1
1
2
2
2
2
3
1
1
2
2
2
2
2
3
1
3
2
2
3
2
2
1
2
3
2
2
2
3
3
3
2
1
2
3
2
2
1
2
2
3
3
3
1
2
2
3
2
2
3
1
2
2
2
3
3
3
2
1
1
3
3
3
3
3
2
2
3
2
2
3
2
2
2
3
1
3
2
2
1
3
1
1
3
2
3
3
2
2
2
1
2
2
3
2
3
1
1
3
2
2
3
3
3
2
2
2
2
1
1
1
1
2
1
2
3
2
2
3
3
2
3
1
2
3
1
2
3
2
2
1
2
3
3
2
2
3
1
2
3
2
3
1
2
2
3
2
1
3
1
3
3
2
3
3
2
2
2
2
1
2
2
3
2
2
1
3
1
2
3
3
2
1
1
2
2
2
2
2
3
3
1
1
2
2
3
2
2
1
2
3
3
2
2
3
2
2
1
1
3
1
2
1
2
2
3
3
1
1
2
3
2
1
2
2
3
1
1
2
2
2
3
2
1
2
3
2
3
2
3
2
2
2
2
2
2
2
2
3
2
1
3
3
3
1
2
1
2
3
3
3
2
3
2
2
3
3
2
1
3
3
3
3
2
3
2
3
2
3
3
3
3
2
3
3
1
3
2
2
2
2
2
2
2
2
1
2
2
1
2
1
2
1
2
2
1
1
2
2
1
3
1
2
2
2
2
3
3
1


# HERE IT ENDS, rest is just few trials

In [2]:
WholeDf.created_at

dateparse = lambda x: WholeDf.created_at.strptime(x, '%Y-%m-%d %H:%M:%S')
#parse_dates=['column name']
datecreated = WholeDf

datecreated = pd.to_datetime(WholeDf.created_at, format='%Y-%m-%d')

In [4]:
WholeDf['date'] = pd.DatetimeIndex(datecreated).normalize()

In [5]:
WholeDf

,Rating,STOCK,created_at,id,text,date
0,4.0,AAPL,2017-10-13 14:20:00,9.188438e+17,Apple cofounders Woz U aims to train tech work...,2017-10-13
1,1.0,AAPL,2017-10-19 23:48:52,9.211613e+17,Anemic iPhone 8 demand drags Apple shares lower,2017-10-19
2,2.0,AAPL,2017-10-11 14:17:00,9.181180e+17,Apple CEO Tim Cook says rumored AR glasses won...,2017-10-11
3,4.0,AAPL,2017-10-20 22:45:00,9.215080e+17,Apple CEO Tim Cook talks aout new Chicago flag...,2017-10-20
4,3.0,AAPL,2017-10-12 16:58:00,9.185210e+17,Apple CEO Tim Cook Learn to code its more impo...,2017-10-12
5,2.0,AAPL,2017-10-23 20:37:00,9.225630e+17,Apple cofounder Steve Wozniak iPhone X is the ...,2017-10-23
6,2.0,AAPL,2017-10-24 03:59:00,9.226740e+17,Apple cofounder Steve Wozniak iPhone X is the ...,2017-10-24
7,3.0,AAPL,2017-10-25 13:31:00,9.231800e+17,Apple cofounder Steve Wozniak Therexe2x80x99s ...,2017-10-25
8,4.0,AAPL,2017-10-14 04:20:00,9.190552e+17,Apple cofounders Woz U aims to train tech work...,2017-10-14
9,4.0,AAPL,2017-10-24 02:31:00,9.226520e+17,Apple COO Jeff Williams on tech and health car...,2017-10-24


In [4]:
for index,sheet_name in enumerate(SheetsToParse):
    if index == 0:
        WholeDf = df.parse(sheet_name)
        WholeDf["STOCK"]= sheet_name
    else:
        tempDf = df.parse(sheet_name)
        tempDf["STOCK"]= sheet_name
        WholeDf= WholeDf.append(tempDf) #read a specific sheet to DataFrame
        

In [5]:
import numpy as np
np.where((WholeDf['Rating'] == WholeDf['Rating']) == True)

(array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
         13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
         26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
         39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
         52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
         65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
         78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
         91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
        104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
        117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
        130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
        143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
        156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
        169, 170, 171, 172, 173, 174, 175, 176, 177

In [7]:
WholeDf['text'] = WholeDf['text'].apply(lambda x: removeURLFromText(x))

In [8]:
WholeDf['text']

0     Apple cofounders Woz U aims to train tech work...
1      Anemic iPhone 8 demand drags Apple shares lower 
2     Apple CEO Tim Cook says rumored AR glasses won...
3     Apple CEO Tim Cook talks aout new Chicago flag...
4     Apple CEO Tim Cook Learn to code its more impo...
5     Apple cofounder Steve Wozniak iPhone X is the ...
6     Apple cofounder Steve Wozniak iPhone X is the ...
7     Apple cofounder Steve Wozniak Therexe2x80x99s ...
8     Apple cofounders Woz U aims to train tech work...
9     Apple COO Jeff Williams on tech and health car...
10    Apple COO Jeff Williams on tech and health car...
11    Apple design chief Jony Ive says he despises m...
12    Apple has agreed to terms with Bain Capital in...
13    Apple hit with trademark lawsuit over iPhone X...
14    Apple reportedly reduced the accuracy of the i...
15    Apple was the worlds 70th largest company a de...
16    Apple was the worlds 70th largest company a de...
17    Apple was the worlds 70th largest company 

In [12]:
WholeDfNonNULL =WholeDf[WholeDf.Rating == WholeDf.Rating]

In [16]:
a = WholeDf.groupby(by="Rating")

In [18]:
a.all

<function pandas.core.groupby._GroupBy._make_wrapper.<locals>.wrapper>

In [ ]:
a.count()